# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
DATA_FOLDER = 'Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [ ]:
# Write your answer here

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

### Libraries used

Glob library is used in order to leverage regular expression capabilities when reading multiple data with similar name.

In [ ]:
import pandas as pd
import numpy as np
import glob, os

### Importing the spreadsheets into a single `DataFrame`

Individual `DataFrames` are read by matching the expression of the file name (*MID#*) and are read into list of dataframes. They are concatenated into a single dataframe `MIDs`.

No header exists in the dataset, so the data is read with appropriate function parameters. In order to be able to track the provenance of the data for the metadata, an additional column *Barcode* is added.

In [ ]:
dataframes = []

# Iterating through all the files satisfying the "regex"
for file in glob.glob(pathname=DATA_FOLDER+'/microbiome/MID*.xls'):
    
    # Reading the file
    dataframe = pd.read_excel(file, 'Sheet 1', header=None)
    
    # Defining the barcode attribute so we can track back rows to metadata
    dataframe['BARCODE']=os.path.basename(file).split(".")[0]

    # Giving semantics to columns
    dataframe.columns = ['Taxon','Count', 'BARCODE']
    dataframe.index.name = 'No.'
    
    # Appending the read dataframe to the list for later concatenation
    dataframes.append(dataframe)

# Concatenating dataframes from the list into single dataframe
MIDs = pd.concat(dataframes, axis=0)

In [ ]:
MIDs.head()

### Combining the DataFrame with metadata

Reading the metadata, with *BARCODE* as a natural choice for the key, as it is supposed to be unique.

In [ ]:
metadata = pd.read_excel(DATA_FOLDER+'/microbiome/metadata.xls', index_col=0)

In [ ]:
metadata

Executing left join on MIDs DataFrame to make a connection with metadata DataFrame. The join is performed on the *BARCODE* column.

In [ ]:
finalDF = MIDs.join(metadata, on='BARCODE')
finalDF.head()

### Index uniqueness

Let us assert if the existing choice to index on the occurence when reading the separate data file results in an unique index:

In [ ]:
finalDF.index.is_unique

It could have been guessed that for uniqueness number of row of each file would not be appropriate, since we have concatenated several such files. Another strategy would be to add additional information unique to the each file, such as *BARCODE*.

In [ ]:
finalDF_unique = finalDF.copy()
finalDF_unique.index = finalDF.index.astype(str) + ' ' + finalDF.BARCODE
finalDF_unique.index.name = 'id'

finalDF_unique.index.is_unique

In [ ]:
finalDF_unique.head()

#### Remark on index uniqueness

Different approaches could be taken in unique index selection. One of the options could have been to use *Taxon* as a primary index. It would still result in a non-unique index when different files are combined in a single DataFrame since same taxons are present in different measurements. On the same note, concatenating the *BARCODE* value would result in an unique index.k

With current approach the row identifier length is kept small (6 characters), while with the aforementioned option such size would both vary and be significantly larger, potentially impacting performance.

In [ ]:
finalDF_unique_h1 = finalDF.copy()
finalDF_unique_h1 = finalDF_unique_h1.set_index(["Taxon","BARCODE"])

finalDF_unique_h1.head()

In [ ]:
finalDF_unique_h1.index.is_unique

Another option which would bring more semantic and group the data would be to perform hierarchical indexing, based on *BARCODE* as level 0, and *Taxon* as level 1 index:

In [ ]:
finalDF_unique_h2 = finalDF.copy()
finalDF_unique_h2 = finalDF_unique_h2.set_index(["BARCODE","Taxon"])

finalDF_unique_h2

In [ ]:
finalDF_unique_h2.index.is_unique

### Finally, handling NaN values

For this purpose we will use `finalDF_unique` DataFrame, but the logic remains the same whichever type of indexing we pick.

In [ ]:
finalDF_unique.fillna('unknown', inplace=True)

finalDF_unique.head()

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np

In [ ]:
# loading data from the xls file
titanic_data = pd.read_excel(DATA_FOLDER+'/titanic.xls')

### Exercise 1

Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 

#### Type and value range

We use **dtypes** parameter of the DataFrame in which we loaded the xls to describe the types of the attributes

We use the **describe()** method for the value range with the following assumptions:

>if the describe() method returns a Series that has a *min* and a *max* entry we use those limits as a range

>otherwise

>if the describe() method returns a Series that has an *unique* entry then we assume that the type of the attribute is not a number but rather an object so mapping all the unique entries to an integer id starting from one, we can have a range from 1 -> # of unique values

In [ ]:
# dtypes parameter returns information regarding the type of each Series in a DataFrame
titanic_data.dtypes

In [ ]:
# variables we use to collect the min and max values for each attribute
min_ranges = []
max_ranges = []
for c in titanic_data.columns:
    # obtain the description of current column
    column_description = titanic_data[c].describe()
    if 'min' in column_description and 'max' in column_description: 
        min_ranges.append(column_description['min'])
        max_ranges.append(column_description['max'])
    elif 'unique' in column_description:
        min_ranges.append('1')
        max_ranges.append(column_description['unique'])
        
# we put the data obtained into a DataFrame indexed by the column names
# which is automatically indexed by the column names given that 
# titanic_data.dtypes returns a series
df = pd.DataFrame({
    'type':titanic_data.dtypes,
    'min range':min_ranges,
    'max range':max_ranges,
})[['type', 'min range', 'max range']]
df

#### Categorical data

**pclass** column represents the three travel classes and can be a nominal categorical variable. Even though the data in this column already has type int64 and the set of values (1,2,3). The number of possible values is much smaller than the number of data entries so according to the docs, it will use less space. Indeed, after some testing, the number of bytes post conversion is smaller than pre. Therefore it would make sense (space, eficiency, and semantically wise) to convert it to categorical

**sex** column is populated for every data entry in the dataset with either "male"/"female". In this case, comparing to the one above, there is an even better improvement regarding the space if we convert the column to categorical.

**survived** column has values 1 and 0 so it makes sense to be nominal categorical variable

**embarked** column denotes one of the three ports of embarkation S, C and Q (here we assume that these are all the ports of embarkation because there are 2 missing values) so it can be a nominal categorical variable

**sibsp** column represents the number of siblings/spouses a passenger had on board. It takes values in the set {0,1,2,3,4,5,8} but this set is not well defined therefore we choose not to consider it categorical.

**cabin** column has a rather small number of unique values 186/295 so semantically cannot be said it fully represents a category. The set being rather small, we choose not to consider it categorical

**boat** column with 28/486 unique values can definitely be considered a nominal categorical variable 

**home.dest** column contains string representations of multiple cities, states and countries, common to multiple passengers. Casting it to categorical is not only a reasonable idea but also an effiency improvement  

We assumed that the **ticket column** representing the ticket number is individual, even though the number of unique values is smaller than the number of passengers, therefore it does not make sense to be a category

In [ ]:
# testing the memory used by a Series with a values set of (1,2,3) before and after casting to category
print('Number of unique values:', len(titanic_data.pclass.value_counts()))
print('Type of data:', titanic_data.pclass.dtype)
print('Nr of bytes before cast:', titanic_data.pclass.nbytes)
print('Nr of bytes after  cast:', titanic_data.pclass.astype('category').nbytes)

In [ ]:
#categorical_columns = ['pclass', 'sex', 'survived', 'embarked', 'boat', 'home.dest']
categorical_columns = ['pclass', 'sex', 'embarked', 'boat', 'home.dest']
for cat_col in categorical_columns:
    titanic_data[cat_col] = titanic_data[cat_col].astype('category')

In [ ]:
titanic_data['boat'].dtype

### Exercise 2: Histograms -> would it make more sense for the values to be normalized?

Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 

In [ ]:
# libraries used for plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [ ]:
''' 
Function that makes it easy to create a bar plot

Takes a Series as data 
'''
def bar_plot(data, title, labels_rotation_angle=0):
    fig, ax = plt.subplots()
    ax.set_title(title + ' bar plot', fontsize=15, fontweight='bold')
    sns.barplot(data.keys(), data, ax=ax)
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=labels_rotation_angle)
    plt.show()

#### Travel class bar plot 

For the travel class it makes sense to use a bar plot for visualization because the data has a discrete value set of {1,2,3} therefore we do not accumulate the values over some intervals so that a histogram to be necessary

In [ ]:
travel_class_counts = titanic_data.pclass.value_counts().sort_index()
travel_class_counts.index = ['First class', 'Second class', 'Third class']
bar_plot(travel_class_counts, 'Travel class')

In [ ]:
# seaborn one-liner
sns.factorplot(x='pclass', data=titanic_data, kind='count')

#### Embarkation point bar plot 

For the embarkation point it makes sense to use again a bar plot because of the same reason as for the travel class, only here the values set is {S, C, Q}

In [ ]:
embarkation_port_counts = titanic_data.embarked.value_counts().sort_index()
embarkation_port_counts.index = ["Cherbourg", "Queenstown", "Southampton"]
bar_plot(embarkation_port_counts, 'Embarkation point')

In [ ]:
titanic_data.columns

In [ ]:
# seaborn one-liner
sns.factorplot(x='embarked', data=titanic_data, kind='count')

#### Passengers sex bar plot

For the passengers sex is reasonable as well to us a bar plot given that we have only two discrete values {female, male}

In [ ]:
passengers_sex_counts = titanic_data.sex.value_counts()
bar_plot(passengers_sex_counts, 'Passenger sex')

In [ ]:
# seaborn one-liner
sns.factorplot('sex', data=titanic_data, kind='count')

#### Passengers age histogram  ---- !!**Must make assumptions for the age < 0**

Given that we want to visualize the age in discrete decade intervals, meaning that multiple values will vote for the same bar, we will use a histogram. 


For this attribute we ignore the NaN values.

In [ ]:
'''
Funciton to easily plot a histogram
Expects a Pandas Series and an array corresponding to the margins of the bins 
'''
def histogram_plot(data, bins,title=""):
    fig, ax = plt.subplots()
    ax.set_title(title + ' histogram', fontsize=15, fontweight='bold')
    plt.hist(data, bins)
    plt.xlabel('age')
    plt.ylabel('# of passengers')
    plt.show()

In [ ]:
# first approach -> using histogram built-in functions and specifying the bins
passengers_age = titanic_data.age[pd.notnull(titanic_data.age)]
histogram_plot(passengers_age, bins=[0,10,20,30,40,50,60,70,80,90], title='Passengers age')

In [ ]:
# second approach -> discretization of data using pandas cut function
age_by_decade = pd.cut(passengers_age, [0, 10, 20, 30, 40, 50, 60, 70, 80, 90], right=False).value_counts()
bar_plot(age_by_decade, 'Age by decade', labels_rotation_angle=90)

### Exercise 3 
Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.

In order to do this, we must first determine the number of passengers per cabin floor. This number is not accurate with respect to reality given that not every passenger's data entry contains information about the cabin floor.

We assume that the decks were index by letters {A, B, C, D, E, F, G} https://www.encyclopedia-titanica.org/titanic-deckplans/.
We assume that the first character in the cabin column corresponds to the cabin's deck.

Cases that we take care of:
>Passengers with multiple cabins. Solution: we observe that the same multiple-cabins values correspond to multiple passengers. Given that we are interested in number of passengers per floor, we simply add one to the corresponding floor for each of them. The only problem could be that a passenger has cabins on multiple floors. So we check for these cases and find that there are not. Deck F has multiple sections with different identification letters as described lower so we do not consider them multiple floors.
>
>Cabin data with value T. Solution: we consider these values wrong and replace them with np.nan because T does not correspond to any deck as can be seen in https://www.encyclopedia-titanica.org/titanic-deckplans/
>
>Cabin data with values that start with F correspond to the F-Deck which has several sections (as can be seen in the F-deck plan https://www.encyclopedia-titanica.org/titanic-deckplans/f-deck.html) like R, G, E, C, J, H, hence the values like "F G63" 

In [ ]:
def check_multiple_floors(cabin_entry):
    cabins = cabin_entry.split(' ')
    # extract the deck letter
    for i in range(len(cabins)):
        cabins[i] = cabins[i][0]
    # verify if all values are equal
    return not(cabins[1:] == cabins[:-1])

for cabin_entry in titanic_data.cabin:
    if cabin_entry is not np.nan and (check_multiple_floors(cabin_entry)):
        if cabin_entry.startswith('F'):
            print("This format specifies a section of the F-deck:", cabin_entry)
        else:
            print(cabin_entry, ' has cabins on multiple floors')

In [ ]:
# replace T values with np.nan
titanic_data['cabin'] = titanic_data.cabin.map(lambda cabin: np.nan if cabin == 'T' else cabin)

In [ ]:
# get a look at the values on F-deck
titanic_data[titanic_data.cabin.str[0] == 'F'].cabin

In [ ]:
import matplotlib.pyplot as plt
'''
Simple function to make it ease to create pie charts
data: array-like
labels: list
'''
def pie_chart(data, labels, title, start_angle, colours=None):
    plt.pie(data, labels=labels, autopct='%1.1f%%', startangle=start_angle, )
    plt.title(title, fontsize=15, fontweight='bold')
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

cabins_per_floor = titanic_data.cabin.str[0].value_counts().sort_index()

piechart_title = "Proportions of passengers by cabin floor"
pie_chart(cabins_per_floor.values, cabins_per_floor.index, piechart_title, 90)

### Survival proportions by travel class

For each travel class, calculate the proportion of the passengers that survived. Present your results in pie charts.

In [ ]:
# check if we have survival data for all passengers
titanic_data.survived.notnull().all()

In [ ]:
class_statistics = titanic_data.groupby(titanic_data.pclass)['survived'].agg(['sum', 'count'])
class_statistics['count'] = class_statistics['count'] - class_statistics['sum']
class_statistics.columns = ['survived', 'did_not_survive']
for i in range(len(class_statistics)):
    pie_chart(class_statistics.iloc[i], class_statistics.columns, 'Proportion of surviving passengers for class {}'.format(i+1), 90)

### Survival proportions by class and sex
Calculate the proportion of the passengers that survived by travel class and sex. Present your results in a single histogram.


In [ ]:
# calculate the proportions grouping the date by class and sex, then aggregating over survived column
# using sum to get the number of surviving passengers and count to get the total number of passengers
class_sex_statistics = titanic_data.groupby(['pclass', 'sex'])['survived'].agg(['sum', 'count'])
class_sex_statistics.columns = ['survived', 'count']
class_sex_statistics['survival_rate'] = class_sex_statistics['survived']/class_sex_statistics['count']

In [ ]:
statistics_reindexed = class_sex_statistics.reset_index()

In [ ]:
sns.factorplot(x='pclass', y='survival_rate', hue='sex',
               data=statistics_reindexed, kind='bar', size=6)

In [ ]:
#seaborn one-liner
sns.factorplot(x="pclass", y="survived", hue="sex", data=titanic_data,
               size=6,kind="bar")

### Age categories
Create 2 equally populated age categories and calculate survival proportions by age category, travel class and sex. Present your results in a DataFrame with unique index.



>For this task it was decided that the NaN values of the attribute age to be excluded.

To divide the data in two age categories we choose to use the **qcut** method because it creates two meaningful categories (<= 28 and >28) with the number of values as close as possible. We could have splitted the data ourselves in two equally populated age categories but we would loose their meaningfulness.

In [ ]:
titanic_clean_age = titanic_data[pd.notnull(titanic_data.age)].copy()
titanic_clean_age.reset_index(inplace=True)

In [ ]:
categ = pd.qcut(titanic_clean_age.age, 2)

In [ ]:
titanic_clean_age['age_cat'] = categ

In [ ]:
cat_stats = titanic_clean_age.groupby(['age_cat', 'pclass', 'sex'])['survived'].agg(['sum','count'])
cat_stats.columns=['survived', 'total']
cat_stats['proportions'] = cat_stats['survived']/cat_stats['total']

In [ ]:
cat_stats

In [ ]:
cat_stats.index.is_unique